# Vellend's (Moran) Drift Model
## python edition

### Getting started
First we have to import a bunch of libraries to do the things we want to do. I will be using:
- numpy: let's us do vectorized math and has lots of useful functions (I do not endore the numpy "core team" they are assholes)
- matplotlib: let's us make plots (not as good as ggplot, seem like good people)
- seaborn: more plotting options (also not as good as ggplot)
- tqdm: progress bar, these models get slooooow

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from tqdm.notebook import tqdm
%matplotlib inline
%matplotlib widget

## The Moran Model
The Moran model is a population genetic model first proposed by Patrick Moran in 1958. (I know a Pat Moran but he's just a terrible DJ.) In its most basic form, The Moran model tracks the changes in allele frequency due purely to genetic drift. 

We imagine a haploid population that has one of two alleles for some gene, say `A` and `a`. We randomly replace individuals in the population with these two alleles until one completely takes over and the other goes extinct.

It seems to me, after reading this chapter that Vellend's whole idea is based around not just the application of the ideas of population genetic 'fundamental forces' but also population genetic _models_. The Moran model is classic and well understood analytically. This means that if we buy into the idea of the population genetic forces we get some nice results for free. The big shift is that instead of talking about alleles, we talk about species and instead of talking about a population, we talk about a community.

In vellends description, we choose a number of years ($Y$) and for each year we perform the same set of operations a number of times equal to the number of individuals in our community ($N$). These operations are depicted here:

![title](../resources/moran_model.png)

The steps above are labelled with numbers:
1. Chose a random individual to die (maybe shouldn't have used pictured of my best friend!)
2. Chose a random living individual to reproduce
3. Replace the dead individual with a member of the reproducing individual

We perform each of these steps $Y \times N$ times. We can precisely define some of the relevant quantities. We're going to define $S_i$ as the species label of individual $i$. Then we can define the species frequency and some important probabilities.

First, the frequency each species ($s$). This is just a fancy way of saying the number of members of one species divided by the total community size.
\begin{align}
f_s = \frac{\sum_{i=0}^N \delta_{S_i s}}{N}
\end{align}

What is important is the two species case that we consider in this chapter.
\begin{align}
f_1 = \frac{\sum_{i=0}^N \delta_{S_i 1}}{N}; \; \; f_2 = 1 - f_1
\end{align}

In the steps above we draw randomly from the community which means that the probability of an individual of species 1 being chosen for reproduction is really simple.
\begin{align}
P_{birth}(S = 1) = f_1; \;\; P_{birth}(S=2) = 1 - f_1
\end{align}

This is dumb and obvious but later, when we introduce selection it will be much less straight-forward.

## Implementation of the Moran Model
My implementation is very similar to Vellend's though I've changed some names of variables and obviously its in python rather than R. If you are interested in the R code shown in the book it is _not_ available at the link in the text but can be found [here](http://mvellend.recherche.usherbrooke.ca/TOEC.html).

Here is a list of the variables as listed in Vellend as well as the name I've used, and the symbol I've used in the math if appropriate.

| Vellend's R       | Pat's python      | Math symbol       |
| :---------------- | :---------------  | -----------------:|
| `J`               | `n_indiv`         | $N$ |
| `init.1`          | `init_f1`         | $f_1$ |
| `COM`             | `comm`            |             |
| `num.years`       | `n_years`         | $Y$ |
| `freq.1`          | `f1`              | $f_1$ |

He also does some stuff in single lines that I've expanded into multiple lines (hopefully) for clarity.

Finally I defined mine as a function which will allow us to use some little interactive controls.

In [2]:
def moran_model(n_indiv, n_years, init_f1):
    """ Implements the Moran model and simulates it in time. 
        
        Parameters
        ----------
        n_indiv: int
            number of individuals in the community
        n_years: int
            number of potential 'turnovers' of the community
        init_f1: float
            initial frequency of species 1
            
        Return
        ------
        moran: np.array (n_years, 2)
            contains the species frequencies for each year of the simulation """

    # set up an empty array for the simulated frequencies
    # initialize with the given frequency. python counts from 0, sorrryyyy
    moran = np.zeros((n_years, 2))
    moran[0] = np.array([init_f1, 1 - init_f1])

    # get a vector representing the community. vellend calls this COM
    # this just makes a random vector drawn from the initial frequency
    comm = np.random.choice([0, 1], size=n_indiv, replace=True, p=moran[0])

    # now we can do the loop as in vellend. I'm going to write it a bit differently
    # but the idea is basically the same. (yes its slower)
    for year in range(1, n_years):

        # for each year we potentially replace each individual so we can loop
        # over that as well
        for indiv in range(n_indiv):
            # get probability of species 1
            f1 = np.sum(comm == 0) / n_indiv

            # replace one individual with another
            death = np.random.randint(n_indiv)
            birth = np.random.choice([0, 1], p=[f1, 1 - f1])
            comm[death] = birth
        
        # when we're done looping over all of the individuals we can update
        # frequencies for the year (the timescale we care about)
        f1 = np.sum(comm == 0) / n_indiv
        moran[year] = np.array([f1, 1 - f1])

    return moran

### Running the simulations and visualization
This is all very messy looking. Such is plotting in python. I think I'm not really going to explain the point other than to say that the first part draws some hopefully attractive plots and the second part let's us do that interactively with a simple graphical interface.

In [8]:
def draw_simulation(iterations=10, individuals=250, years=50, initial_freq=0.5):
    # the plot bit, this just makes a blank plot
    fig, ax = plt.subplots(ncols=2, figsize=(10,4), sharey=True, gridspec_kw = {'width_ratios':[3, 1]})
    # trajectory labels
    ax[0].set_xlabel('Years')
    ax[0].set_ylabel('Species 1 frequency')
    ax[0].set_ylim([0, 1.01])
    # distribution labels and bins
    hist_bins = np.arange(0, 1.1, 0.1)
    ax[1].set_xlabel('Count')
    ax[1].set_xlim([0, iterations])

    # we're going to plot a distribution too need an array for it
    dist = np.zeros(iterations)

    # we run the simulation and draw a trajectory
    for i in tqdm(range(iterations)):
        simulation = moran_model(individuals, years, initial_freq)
        ax[0].plot(range(years), simulation[:, 0], c='C0')
        # add final freq to our dist array
        dist[i] = simulation[-1, 0]

    # plot the distribution too
    ax[1].hist(dist, bins=hist_bins, orientation='horizontal')
    ax[1].axhline(np.mean(dist), linestyle='--')


    plt.tight_layout()    
    plt.show()

# set up the interface
widgets.interact_manual(draw_simulation, iterations=(5, 50, 5), individuals=(10, 100, 10), years=(5, 200, 5),
                  initial_freq=(0.0, 1.0, 0.01))

interactive(children=(IntSlider(value=10, description=&#39;iterations&#39;, max=50, min=5, step=5), IntSlider(value=10…

&lt;function __main__.draw_simulation(iterations=10, individuals=250, years=50, initial_freq=0.5)&gt;